In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Load the dataset
file_path = Path('./Resources/Parking_Violations.csv')
parking_data = pd.read_csv(file_path)


/var/folders/tj/b8kc10px4jlb5mfbkfmw3ffw0000gn/T/ipykernel_6173/1872182343.py:3: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  parking_data = pd.read_csv(file_path)


In [3]:
parking_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98493 entries, 0 to 98492
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   OBJECTID              98493 non-null  int64  
 1   TICKET_NUMBER         98493 non-null  int64  
 2   VIOLATION_TYPE_DESC   98493 non-null  object 
 3   ISSUE_DATE            98493 non-null  object 
 4   ISSUE_TIME            98493 non-null  int64  
 5   ISSUING_AGENCY_CODE   98493 non-null  int64  
 6   ISSUING_AGENCY_NAME   98493 non-null  object 
 7   ISSUING_AGENCY_SHORT  98493 non-null  object 
 8   VIOLATION_CODE        98493 non-null  object 
 9   VIOLATION_PROC_DESC   98485 non-null  object 
 10  LOCATION              98493 non-null  object 
 11  PLATE_STATE           98493 non-null  object 
 12  VEHICLE_TYPE          0 non-null      float64
 13  MULTI_OWNER_NUMBER    98493 non-null  int64  
 14  DISPOSITION_CODE      98493 non-null  int64  
 15  DISPOSITION_TYPE   

In [4]:
# Select relevant features and target variable
data = parking_data[['VIOLATION_TYPE_DESC', 'ISSUE_TIME', 'ISSUING_AGENCY_CODE', 'XCOORD', 'YCOORD']].copy()

data

,VIOLATION_TYPE_DESC,ISSUE_TIME,ISSUING_AGENCY_CODE,XCOORD,YCOORD
0,P,657,54,NaN,NaN
1,P,133,59,NaN,NaN
2,P,1020,59,397076.780,135370.390
3,P,1051,59,397328.708,135348.587
4,P,1108,59,397328.637,135237.646
...,...,...,...,...,...
98488,P,1535,15,399383.850,135200.010
98489,P,1638,15,399995.838,135331.512
98490,P,1639,15,399995.838,135331.512
98491,P,1640,15,399995.838,135331.512


In [5]:
# Drop rows with missing values in the selected columns
data.dropna(inplace=True)



In [6]:

# Encode categorical variables
data['VIOLATION_TYPE_DESC'] = data['VIOLATION_TYPE_DESC'].astype('category').cat.codes
data['ISSUING_AGENCY_CODE'] = data['ISSUING_AGENCY_CODE'].astype('category').cat.codes


In [7]:
# Define features (X) and target variable (y)
X = data[['ISSUE_TIME', 'ISSUING_AGENCY_CODE', 'XCOORD', 'YCOORD']]
y = data['VIOLATION_TYPE_DESC']


In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [9]:
# Initialize and train a Random Forest Classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
# Predict on the test data
y_pred = clf.predict(X_test)


In [11]:
# Generate and print the classification report
print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           1       0.99      0.99      0.99     28315
           2       0.97      1.00      0.98        63
           3       0.60      0.54      0.57       890

    accuracy                           0.98     29268
   macro avg       0.85      0.84      0.85     29268
weighted avg       0.97      0.98      0.97     29268



In [16]:

# Example of encoding categorical variables if they are not numeric
label_encoders = {}
for column in ['LOCATION', 'ISSUE_TIME', 'ISSUING_AGENCY_NAME']:
    le = LabelEncoder()
    parking_data[column] = le.fit_transform(parking_data[column])
    label_encoders[column] = le

# Define features (X) and target variable (y)
X = parking_data[['LOCATION', 'ISSUE_TIME', 'ISSUING_AGENCY_NAME']]
y = parking_data['VIOLATION_TYPE_DESC']  # Ensure this is the target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [17]:
from sklearn.linear_model import LogisticRegression

# Initialize the model
log_reg = LogisticRegression()

# Train the model
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)

# Evaluate the model
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Logistic Regression Accuracy: 0.9639907946392311
              precision    recall  f1-score   support

           M       0.00      0.00      0.00         5
           P       0.97      1.00      0.98     28526
           T       0.03      0.03      0.03        80
           V       0.00      0.00      0.00       937

    accuracy                           0.96     29548
   macro avg       0.25      0.26      0.25     29548
weighted avg       0.93      0.96      0.95     29548



/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the model
tree = DecisionTreeClassifier()

# Train the model
tree.fit(X_train, y_train)

# Make predictions
y_pred = tree.predict(X_test)

# Evaluate the model
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Decision Tree Accuracy: 0.9701841072153784


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           M       0.00      0.00      0.00         5
           P       0.98      0.99      0.98     28526
           T       0.96      0.97      0.97        80
           V       0.54      0.52      0.53       937

    accuracy                           0.97     29548
   macro avg       0.62      0.62      0.62     29548
weighted avg       0.97      0.97      0.97     29548



/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
forest.fit(X_train, y_train)

# Make predictions
y_pred = forest.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Random Forest Accuracy: 0.9691349668336267
              precision    recall  f1-score   support

           M       0.00      0.00      0.00         5
           P       0.98      0.99      0.98     28526
           T       0.96      1.00      0.98        80
           V       0.53      0.36      0.43       937

    accuracy                           0.97     29548
   macro avg       0.62      0.59      0.60     29548
weighted avg       0.96      0.97      0.97     29548



/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
from sklearn.svm import SVC

# Initialize the model
svm = SVC()

# Train the model
svm.fit(X_train, y_train)

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate the model
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


SVM Accuracy: 0.965412210640314


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           M       0.00      0.00      0.00         5
           P       0.97      1.00      0.98     28526
           T       0.00      0.00      0.00        80
           V       0.00      0.00      0.00       937

    accuracy                           0.97     29548
   macro avg       0.24      0.25      0.25     29548
weighted avg       0.93      0.97      0.95     29548



/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize the model
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
print("KNN Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


KNN Accuracy: 0.9677812373087857


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           M       0.00      0.00      0.00         5
           P       0.98      0.99      0.98     28526
           T       0.69      0.50      0.58        80
           V       0.53      0.40      0.46       937

    accuracy                           0.97     29548
   macro avg       0.55      0.47      0.51     29548
weighted avg       0.96      0.97      0.97     29548



/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the model
gbc = GradientBoostingClassifier()

# Train the model
gbc.fit(X_train, y_train)

# Make predictions
y_pred = gbc.predict(X_test)

# Evaluate the model
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Gradient Boosting Accuracy: 0.9656829565452822


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           M       0.00      0.00      0.00         5
           P       0.97      1.00      0.98     28526
           T       1.00      0.03      0.05        80
           V       0.59      0.02      0.03       937

    accuracy                           0.97     29548
   macro avg       0.64      0.26      0.27     29548
weighted avg       0.95      0.97      0.95     29548



/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
